## Imports

In [1]:
import copy
from tqdm import tqdm

In [2]:
day = "22"

In [3]:
with open(f"input_j{day}.txt") as f:
# with open("test.txt") as f:
    bricks = f.readlines()
    bricks = list(map(lambda x: x.replace("\n", ""), bricks))

n = len(bricks)
for i in range(n):

    bricks[i] = list(map(
        lambda end: [int(x) for x in end.split(",")],
        bricks[i].split("~")
    ))

    if bricks[i][0][2] > bricks[i][1][2]:
        bricks[i][0].reverse()

    bricks[i] = [bricks[i][0][2]] + bricks[i]

bricks.sort()
for i in range(n):
    bricks[i] = bricks[i][1:]

print(n)

1479


In [4]:
bricks[:3]

[[[0, 0, 1], [0, 0, 1]], [[1, 7, 1], [1, 9, 1]], [[2, 2, 1], [5, 2, 1]]]

## Application

In [5]:
liste_x = [end[0] for row in bricks for end in row]
x_max = max(liste_x)
x_max

9

In [6]:
liste_y = [end[1] for row in bricks for end in row]
y_max = max(liste_y)
y_max

9

In [7]:
liste_z = [end[2] for row in bricks for end in row]
z_max = max(liste_z)
z_max

357

In [8]:
fallen_bricks = [[[-1 for z in range(z_max+1)] for y in range(y_max+1)] for x in range(x_max+1)]

In [9]:
def create_brick(ends):
    if ends[0][0] != ends[1][0]:
        return [
            [x, ends[0][1], ends[0][2]] for x in range(
                min(ends[0][0], ends[1][0]),
                max(ends[0][0], ends[1][0])+1
            )
        ]
    if ends[0][1] != ends[1][1]:
        return [
            [ends[0][0], y, ends[0][2]] for y in range(
                min(ends[0][1], ends[1][1]),
                max(ends[0][1], ends[1][1])+1
            )
        ]
    if ends[0][2] != ends[1][2]:
        return [
            [ends[0][0], ends[0][1], z] for z in range(
                min(ends[0][2], ends[1][2]),
                max(ends[0][2], ends[1][2])+1
            )
        ]
    return [ends[0]]

In [10]:
for i in tqdm(range(n)):
    brick = create_brick(bricks[i])
    # print(brick)
    z_min = 1
    spots = [
        [
            end[0], end[1], z + end[2] - bricks[i][0][2]
        ] for end in brick for z in range(z_min, z_max) if z + end[2] - bricks[i][0][2] <= z_max
    ]
    while not all([fallen_bricks[spot[0]][spot[1]][spot[2]] == -1 for spot in spots]):
        z_min += 1
        spots = [
            [
                end[0], end[1], z + end[2] - bricks[i][0][2]
                ] for end in brick for z in range(z_min, z_max) if z + end[2] - bricks[i][0][2] <= z_max
            ]
    spots = [[end[0], end[1], z_min + end[2]- bricks[i][0][2]] for end in brick]
    for spot in spots:
        if len(spots) == 0:
            print(i)
        fallen_bricks[spot[0]][spot[1]][spot[2]] = i

100%|██████████| 1479/1479 [00:45<00:00, 32.57it/s]


In [11]:
# fallen_bricks

In [ ]:
compteur = 0
compteur_casses = 0

for i in tqdm(range(n)):

    test_fallen_bricks = copy.deepcopy(fallen_bricks)
    brick = create_brick(bricks[i])
    above_bricks = set()
    for x in set([end[0] for end in brick]):
        for y in set([end[1] for end in brick]):
            for z in range(z_max+1):
                if fallen_bricks[x][y][z] == i:
                    test_fallen_bricks[x][y][z] = -1
                    for z_above in range(z+1, z_max+1):
                        if fallen_bricks[x][y][z_above] not in [-1, i]:
                            above_bricks.add(fallen_bricks[x][y][z_above])

    booleen = True
    # for j in above_bricks:
    for j in range(min(above_bricks), n):
        brick = create_brick(bricks[j])
        # print(j, brick)
        z_min = 1
        z_max_j = min([z for z in range(z_max+1) if test_fallen_bricks[brick[0][0]][brick[0][1]][z] == j])
        spots = [
            [
                end[0], end[1], z + end[2] - bricks[j][0][2]
                ] for end in brick for z in range(z_min, z_max_j+1) if z + end[2] - bricks[j][0][2] <= z_max
            ]
        while not all([test_fallen_bricks[spot[0]][spot[1]][spot[2]] in [-1, j] for spot in spots]):
            z_min += 1
            spots = [
                [
                    end[0], end[1], z + end[2] - bricks[j][0][2]
                    ] for end in brick for z in range(z_min, z_max_j+1) if z + end[2] - bricks[j][0][2] <= z_max
                ]

        if z_min < z_max_j:

            booleen = False
            compteur_casses += 1

            spots_to_add = [[end[0], end[1], z_min + end[2]- bricks[j][0][2]] for end in brick]
            spots_to_remove = [[end[0], end[1], z_max_j + end[2]- bricks[j][0][2]] for end in brick]
            for spot in spots_to_remove:
                test_fallen_bricks[spot[0]][spot[1]][spot[2]] = -1
            for spot in spots_to_add:
                test_fallen_bricks[spot[0]][spot[1]][spot[2]] = j

    if booleen:
        # print(i)
        compteur += 1


In [ ]:
compteur  # Part 1

In [ ]:
compteur_casses  # Part 2